In [1]:
#import geopandas as gp
#shp = gp.GeoDataFrame.from_file('London_Borough_Excluding_MHW.dbf')
#borough_shapes=shp.loc[:,['NAME','geometry']]
#print(borough_shapes)

                           NAME  \
0             Chessington South   
1        Tolworth and Hook Rise   
2                    Berrylands   
3                     Alexandra   
4                      Beverley   
5                   Coombe Hill   
6    Chessington North and Hook   
7                 Surbiton Hill   
8                    Old Malden   
9                    St. Mark's   
10                        Grove   
11                      Canbury   
12                     Norbiton   
13                  Coombe Vale   
14                    St. James   
15                        Tudor   
16                Coulsdon East   
17         Selsdon and Ballards   
18                Coulsdon West   
19                       Waddon   
20                       Kenley   
21                       Purley   
22                  Sanderstead   
23                   Heathfield   
24                    Fairfield   
25                  Broad Green   
26                West Thornton   
27                Be

In [7]:
import geopandas as gp
shp = gp.GeoDataFrame.from_file('London_Ward_CityMerged.shp')
print(shp["NAME"][0],": ",shp["geometry"][0])
#print(ward_shapes)


Chessington South :  POLYGON ((516401.6 160201.8, 516407.3 160210.5, 516413.3 160217.4, 516419.9 160224.2, 516427.9 160231.3, 516453.8 160253, 516472.8 160275.3, 516486.6 160296.2, 516493.8 160310.2, 516500.9 160324.2, 516507.3 160343.6, 516507.2 160358.3, 516502.3 160395.3, 516500.3 160421.5, 516491.1 160483.9, 516486.7 160493.9, 516472.7 160521.7, 516470.2 160556.7, 516468.9 160567.2, 516461.7 160590.2, 516459.2 160610.3, 516459 160620.9, 516459.1 160632, 516462.4 160702.2, 516462.5 160741.9, 516467.2 160775.7, 516471.3 160798, 516466.7 160810.1, 516473.7 160853.1, 516474.5 160882.9, 516477.6 160904.4, 516487.3 160940.6, 516487.3 160973.9, 516485.3 161015.6, 516485.7 161060.1, 516494 161156.5, 516502.3 161223.8, 516503.2 161279.8, 516501.4 161318.8, 516501.6 161361.4, 516500.8 161380.4, 516500 161398.8, 516493.3 161438.8, 516491.6 161452.4, 516495.1 161513.2, 516503.3 161546.7, 516518.2 161621.8, 516519.3 161624.8, 516531.1 161655.2, 516540.5 161685.7, 516552.2 161756.6, 516555.8 161

In [3]:
import shapefile
import pyproj
sf = shapefile.Reader("London_Ward_CityMerged.prj")
wgs_shp = shapefile.Writer(shapefile.POLYGON)
fields = sf.fields
wgs_fields = wgs_shp.fields
for name in fields:
    if type(name) == "tuple":
        continue
    else:
        args = name
        wgs_shp.field(*args)
records = sf.records()
for row in records:
    args = row
    wgs_shp.record(*args)
geom = sf.shapes()
input_projection =  pyproj.Proj(init="epsg:27700")
output_projection =  pyproj.Proj(init="epsg:4326")
for feature in geom:
    # if there is only one part
    if len(feature.parts) == 1:
        # create empty list to store all the coordinates
        poly_list = []
        # get each coord that makes up the polygon
        for coords in feature.points:
            x, y = coords[0], coords[1]
            # tranform the coord
            new_x, new_y =  pyproj.transform(input_projection, output_projection, x, y)
            # put the coord into a list structure
            poly_coord = [float(new_x), float(new_y)]
            # append the coords to the polygon list
            poly_list.append(poly_coord)
        # add the geometry to the shapefile.
        wgs_shp.poly(parts=[poly_list])
    else:
        # append the total amount of points to the end of the parts list
        feature.parts.append(len(feature.points))
        # enpty list to store all the parts that make up the complete feature
        poly_list = []
        # keep track of the part being added
        parts_counter = 0

        # while the parts_counter is less than the amount of parts
        while parts_counter < len(feature.parts) - 1:
            # keep track of the amount of points added to the feature
            coord_count = feature.parts[parts_counter]
            # number of points in each part
            no_of_points = abs(feature.parts[parts_counter] - feature.parts[parts_counter + 1])
            # create list to hold individual parts - these get added to poly_list[]
            part_list = []
            # cut off point for each part
            end_point = coord_count + no_of_points

            # loop through each part
            while coord_count < end_point:
                for coords in feature.points[coord_count:end_point]:
                    x, y = coords[0], coords[1]
                    # tranform the coord
                    new_x, new_y = transform(input_projection, output_projection, x, y)
                    # put the coord into a list structure
                    poly_coord = [float(new_x), float(new_y)]
                    # append the coords to the part list
                    part_list.append(poly_coord)
                    coord_count = coord_count + 1
                # append the part to the poly_list
                poly_list.append(part_list)
                parts_counter = parts_counter + 1
            # add the geometry to to new file
            wgs_shp.poly(parts=poly_list)
wgs_shp.save("Ward_London.shp")

In [151]:
import geopandas as gp
import numpy as np

#shell_coords = np.array(polygon.exterior)
#print(shell_coords)
#points=print(polygon.exterior.coords.xy)
#print(points)

def point_in_poly(x,y,poly):
    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

point_x = -0.12580687
point_y = 51.51240222

shp = gp.GeoDataFrame.from_file('Ward_London.shp')

def pip(x,point_y,shp):
    polygons=shp['geometry']
    places=shp['NAME']
    n=0
    for polygon in polygons:
        polygon=np.array(polygon.exterior)
        if point_in_poly(point_x,point_y,polygon):
            location=places[n]
        n=n+1
    return(location)

print(pip(point_x,point_y,shp))

#check
#polygons=shp['geometry']
#places=shp['NAME']
#index=shp[shp['NAME']=="St. James's"].index
#poly=polygons[index[0]]
#polygon=np.array(poly.exterior)
#print(point_in_poly(point_x,point_y,polygon))

True
St. James's
